# Jupyter Notebook to perform face recognition

## Step 1 - Creating Training Data by capturing face

### Importing the libraries

In [ ]:
import cv2
import os
import numpy as np

# Optional Step
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### Loading Haarcascade classifier & creating the function to extract the faces

In [ ]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface.xml')

def face_extractor(img):

    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image

    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)

    if faces == ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face

### Capturing the face & creating data inside a temporary folder

In [ ]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Creating folder to store the files
try:
    os.makedirs('./faces/user/')
except FileExistsError:
    print ('Directory already exists.')

# Collect 100 samples of your face from webcam input
while True:
    ret, frame = cap.read()
   
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)     
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break

cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

## Step 2 - Training the Model from captured images

### Importing some extra libraries

In [ ]:
from os import listdir
from os.path import isfile, join

### Preprocessing the images & creating the dataset

In [ ]:
# Get the training data we previously made
data_path = './faces/user/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

### Training the model

In [ ]:
# Initialize facial recognizer
model  = cv2.face_LBPHFaceRecognizer.create()

# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")

## Step 3 - Creating functions to perform certain tasks

### Function to send Whatsapp message

In [ ]:
def whatsapp_message():
    # importing the library
    import pywhatkit

    # sending the message
    pywhatkit.sendwhatmsg_instantly("+91123456789", "Your Model has been created successfully.")
    print("Whatsapp message send successfully")

### Function to send Email

In [ ]:
def send_email():
    # importing the library
    import smtplib

    # connecting to smtp server
    smtpObj = smtplib.SMTP('smtp.gmail.com',587)
    smtpObj.ehlo()
    smtpObj.starttls()
    smtpObj.login("MYGMAILID@gmail.com", "PASSWORD")

    # sending the email
    smtpObj.sendmail("abc123@gmail.com", "EXAMPLE@gmail.com", 'Subject: Your model has been trained successfully')
    smtpObj.quit()

### Function to provison resources on AWS

In [ ]:
def aws_resources():
    # importing the libraries
    import subprocess

    result = subprocess.run([r'C://Windows//System32//WindowsPowerShell//v1.0//powershell.exe', r'.//aws.ps1'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    print("AWS Resources provisoned successfully")

## Step 4 - Running Facial Recognition

### Loading Haarcascade classifier & creating the function to detect the faces

In [ ]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces == ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

### Capturing the video & started recognizing the face

In [ ]:
# Open Webcam
cap = cv2.VideoCapture(0)

Success = False

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        results = model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 90:
            cv2.imshow('Face Recognition Success', image )
            cv2.putText(image, "Hey Raktim", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
           
            # here put whatever function you want to run as soon as face recognized
            print("Face Recognition Success... Within few seconds your resources will be provisoned on AWS")
            Success = True
            break
         
        else:   
            cv2.putText(image, "Face not recognized", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()

# Running the function to provison the resources
if Success:
    aws_resources()
else:
    print("Program Exited")